# 3. Coding Attention Mechanisms

In this chapter four different variants of attention mechanisms are built:

1. **Simplified self-attention**: To introduce the broader idea. 

2. **Self-attention**: Self-attention with trainable weights that forms the basis of the mechanisms, used in LLMs.

3. **Causal attention**: A type of self-attention that allows a model to consider only previous and current inputs in a sequence, ensuring temporal order during the text generation

4. **Multi-head attention**: An extension of self-attention and causal attention that enables the model to simultaneously attend to information from different representation subspaces.



Pre-LLMs architectures had some problems. For instance, a model that translates text word by word in a fixed order could lead to grammatical errors. Normally is used an *encoder* and *decoder* to adress this problem, where the job of the encoder is to read in and process the text, and the decoder then produces the translated text. Before the advent of transformers, recurrent neural networks (RNNs) were the most popular encoder–decoder architecture for language translation. An RNN is a type of neural network where outputs from previous steps are fed as inputs to the current step. 


<div style="text-align: center;">
    <img src="images/encoder-decoder-rnn.png" alt="Alt text" width="500">
</div>

The big limitation of encoder–decoder RNNs is that the RNN can’t directly access earlier hidden states from the encoder during the decoding phase. Consequently, it relies solely on the current hidden state, which encapsulates all relevant information. This can lead to a loss of context, especially in complex sentences where dependencies might span long distances.

## 3.2 Capturing data dependencies with attention mechanisms

